### Random Forest

I will go through the whole process of creating a machine learning model on the famous Titanic dataset, which is used by many people all over the world. It provides information on the fate of passengers on the Titanic, summarized according to economic status (class), sex, age and survival.

In [2]:
# linear algebra
import numpy as np 

# data processing
import pandas as pd 

# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

Getting the Data

In [3]:
test_df = pd.read_csv("test.csv")
train_df = pd.read_csv("train.csv")

Data Exploration/Analysis

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Above we can see that 38% out of the training-set survived the Titanic. We can also see that the passenger ages range from 0.4 to 80. On top of that we can already detect some features, that contain missing values, like the ‘Age’ feature.

In [6]:
train_df.head(8)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S


Let’s take a more detailed look at what data is actually missing:

In [7]:
total = train_df.isnull().sum().sort_values(ascending=False)
percent_1 = train_df.isnull().sum()/train_df.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(5)

,Total,%
Cabin,687,77.1
Age,177,19.9
Embarked,2,0.2
Fare,0,0.0
Ticket,0,0.0


The Embarked feature has only 2 missing values, which can easily be filled. It will be much more tricky, to deal with the ‘Age’ feature, which has 177 missing values. The ‘Cabin’ feature needs further investigation, but it looks like that we might want to drop it from the dataset, since 77 % of it are missing.

In [8]:
train_df.columns.values

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object)

### Data Preprocessing

First, I will drop ‘PassengerId’ from the train set, because it does not contribute to a persons survival probability. I will not drop it from the test set, since it is required there for the submission.

In [9]:
train_df = train_df.drop(['PassengerId'], axis=1)

In [10]:
# extracting and then removing the targets from the training data 
targets = train_df['Survived']
train_df.drop(['Survived'], 1, inplace=True)

In [11]:
# merging train data and test data for future feature engineering
# we'll also remove the PassengerID since this is not an informative feature
combined = train_df.append(test_df)
combined.reset_index(inplace=True)
combined.drop(['index', 'PassengerId'], inplace=True, axis=1)

In [12]:
#Now let's map the title can bin them
Title_Dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Dona": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [13]:
#Generate a new Title column
combined['Title'] = combined['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
combined['Title'] = combined['Title'].map(Title_Dictionary)

In [14]:
#let's get the median age based on people's gender, Pclass and Title
fill_mean = lambda g: g.fillna(g.mean())
combined['Age'] = combined.groupby(['Sex', 'Title', 'Pclass'])['Age'].apply(fill_mean)

In [15]:
# Name can be dropped now
combined.drop('Name', axis=1, inplace=True)

In [16]:
# removing the title variable
combined.drop('Title', axis=1, inplace=True)

In [17]:
# removing the Cabin variable
combined.drop('Cabin', axis=1, inplace=True)

In [18]:
# removing the Ticket variable
combined.drop('Ticket', axis=1, inplace=True)

In [19]:
#Fill out the missing fare data
combined['Fare'].fillna(combined['Fare'].mean(), inplace=True)

In [20]:
# two missing embarked values - filling them with the most frequent one in the train set
combined['Embarked'].fillna('S', inplace=True)

In [21]:
# encoding in dummy variable
embarked_dummies = pd.get_dummies(combined['Embarked'], prefix='Embarked')
combined = pd.concat([combined, embarked_dummies], axis=1)
combined.drop('Embarked', axis=1, inplace=True)

In [22]:
# mapping gender to numerical one 
combined['Sex'] = combined['Sex'].map({'male':1, 'female':0})

In [23]:
# introducing a new feature : the size of families (including the passenger)
combined['FamilySize'] = combined['Parch'] + combined['SibSp'] + 1
combined.drop('Parch', axis=1, inplace=True)
combined.drop('SibSp', axis=1, inplace=True)

In [31]:
# the training dataset
df_im_input=combined.iloc[:891]
df_im_output=targets

# the test dataset
test_input=combined.iloc[891:]

### Random Forest and K-fold Validation

In [32]:
## k-fold cross validation
def train_validation_split(data, k):
    number_of_rows = data.shape[0]
    number_of_test = int(np.floor(number_of_rows / k))
    list_of_data_index = [i for i in range(number_of_rows)]
    
    result = pd.DataFrame(columns=['train_index','validation_index'], index=[i for i in range(k)])
    
    for i in range(k):
        total_index = np.array([i for i in range(number_of_rows)])
        test_index = np.random.choice(list_of_data_index, size=number_of_test, replace=False)
        train_index = np.delete(total_index, test_index)
        
        for index in test_index:
            list_of_data_index.remove(index)
        
        # print("{0}th iteration:\n".format({i}), "test index\n", test_index, "\ntrain index\n", train_index)
        
        result.iloc[i,0] = np.sort(train_index)
        result.iloc[i,1] = np.sort(test_index)

    return result



def combination_grid(parameters):
    import itertools
    '''
    parameters: a dictionary, e.g. parameters = {'max_depth':[3, 5, 7, 9], 'min_samples_leaf': [1, 2, 3, 4]}
    '''
    # https://www.codegrepper.com/code-examples/delphi/python+list+of+lists+all+combinations
    combinations = list(itertools.product(*list(parameters.values())))
    df = pd.DataFrame(combinations, columns = parameters.keys()) 
    
    return df

In [33]:
# spliting 6-fold train_validation
split = train_validation_split(df_im_input, 6)
split

,train_index,validation_index
0,"[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[4, 11, 18, 20, 30, 42, 47, 52, 53, 73, 74, 77..."
1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15, 16...","[0, 10, 14, 26, 27, 29, 40, 45, 46, 54, 59, 65..."
2,"[0, 2, 3, 4, 7, 8, 9, 10, 11, 13, 14, 15, 16, ...","[1, 5, 6, 12, 22, 24, 25, 36, 38, 49, 50, 57, ..."
3,"[0, 1, 2, 4, 5, 6, 8, 10, 11, 12, 13, 14, 15, ...","[3, 7, 9, 16, 19, 28, 31, 32, 33, 43, 56, 60, ..."
4,"[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 14, 15...","[8, 13, 35, 41, 48, 58, 64, 69, 89, 95, 116, 1..."
5,"[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[2, 15, 17, 21, 23, 34, 37, 39, 44, 51, 55, 62..."


In [26]:
# make grid of paramters

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 40, stop = 1000, num = 13)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

combination_grid = combination_grid(random_grid)
combination_grid

,n_estimators,max_features,max_depth,min_samples_split,min_samples_leaf,bootstrap
0,40,auto,10,2,1,True
1,40,auto,10,2,1,False
2,40,auto,10,2,2,True
3,40,auto,10,2,2,False
4,40,auto,10,2,4,True
...,...,...,...,...,...,...
7717,1000,log2,110,10,1,False
7718,1000,log2,110,10,2,True
7719,1000,log2,110,10,2,False
7720,1000,log2,110,10,4,True


In [27]:
# apply k-fold validation to the 1st combination of parameters
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc

auc_list = []
for k in range(split.shape[0]):
    train_data = df_im_input[df_im_input.index.isin(split.iloc[k,0])]
    validation_data = df_im_input[df_im_input.index.isin(split.iloc[k,1])]
    train_target = df_im_output[df_im_output.index.isin(split.iloc[k,0])]
    validation_target = df_im_output[df_im_output.index.isin(split.iloc[k,1])]

    rf = RandomForestClassifier(n_estimators=combination_grid.iloc[0,0],
                                max_features=combination_grid.iloc[0,1],
                                max_depth=combination_grid.iloc[0,2],
                                min_samples_split=combination_grid.iloc[0,3],
                                min_samples_leaf=combination_grid.iloc[0,4],
                                bootstrap=combination_grid.iloc[0,5],
                                random_state = 2345)
    
    rf.fit(train_data, train_target)
    
    # preds=rf.predict(train_data)        
    preds=rf.predict(validation_data)

    preds_probabilities = rf.predict_proba(validation_data)
    pred_probs = preds_probabilities[:, 1]
    
    [fpr, tpr, thr] = roc_curve(validation_target, pred_probs)
    auc_ = auc(fpr, tpr)
    auc_list.append(auc_)

auc_avg = sum(auc_list) / len(auc_list) 
print(auc_avg)
# combination_grid['avg_auc'][0] = auc_avg

0.878398641649731


In [28]:
# loop whole combination grid

combination_grid['avg_auc'] = np.zeros((combination_grid.shape[0],1))

for i in range(combination_grid.shape[0]):
    auc_list = []
    for k in range(split.shape[0]):
        train_data = df_im_input[df_im_input.index.isin(split.iloc[k,0])]
        validation_data = df_im_input[df_im_input.index.isin(split.iloc[k,1])]
        train_target = df_im_output[df_im_output.index.isin(split.iloc[k,0])]
        validation_target = df_im_output[df_im_output.index.isin(split.iloc[k,1])]

        rf = RandomForestClassifier(n_estimators=combination_grid.iloc[i,0],
                                max_features=combination_grid.iloc[i,1],
                                max_depth=combination_grid.iloc[i,2],
                                min_samples_split=combination_grid.iloc[i,3],
                                min_samples_leaf=combination_grid.iloc[i,4],
                                bootstrap=combination_grid.iloc[i,5],
                                random_state = 2345)
        
        rf.fit(train_data, train_target)
    
        preds=rf.predict(validation_data)
        preds_probabilities = rf.predict_proba(validation_data)
        pred_probs = preds_probabilities[:, 1]
    
        [fpr, tpr, thr] = roc_curve(validation_target, pred_probs)
        auc_ = auc(fpr, tpr)
        auc_list.append(auc_)

    auc_avg = sum(auc_list) / len(auc_list)
    combination_grid.iloc[i,-1] = auc_avg
    print(auc_avg)

0.878398641649731
0.8697696868338474
0.8771542387217203
0.8734887618114368
0.876219858283612
0.8762959093424287
0.8775789995141404
0.872484710344264
0.8774030987242917
0.8787264823427243
0.876219858283612
0.8762959093424287
0.879191558044132
0.8776637281336543
0.881320753213255
0.8821722978252556
0.8781367232204591
0.875940748921165
0.8656402977681562
0.8450169269625324
0.8808443786650155
0.8716606498872689
0.8777034521111751
0.8754916895190229
0.8781249944380236
0.8650352057524998
0.8807364898356692
0.8729660730415062
0.8777034521111751
0.8754916895190229
0.8788884975200563
0.8766048209524326
0.880886452226207
0.8779128686519592
0.8787377338028676
0.8791173270176676
0.8653847637611801
0.8461147682746669
0.8810223622484782
0.8716554200257338
0.8777034521111751
0.8756448757935327
0.8779682855846319
0.8649650533104761
0.880441163829993
0.8730603324267348
0.8777034521111751
0.8756448757935327
0.8790493461641032
0.876541599793979
0.8809516200333104
0.8778515941421551
0.8787377338028676
0.8

0.8778796341446569
0.8732633981405217
0.8800155547181596
0.880412402019882
0.8704238820321134
0.8660486177088957
0.8788790401661207
0.875919163980539
0.8800155547181596
0.880412402019882
0.8800694028590277
0.8733056643209233
0.8831647294669978
0.8795372564255407
0.8765213754816862
0.8780181722940377
0.8678229772642311
0.8463017138581114
0.8778796341446569
0.8732633981405217
0.8800155547181596
0.880412402019882
0.8704238820321134
0.8660486177088957
0.8788790401661207
0.875919163980539
0.8800155547181596
0.880412402019882
0.8800694028590277
0.8733056643209233
0.8831647294669978
0.8795372564255407
0.8765213754816862
0.8780181722940377
0.8678229772642311
0.8463017138581114
0.8778796341446569
0.8732633981405217
0.8800155547181596
0.880412402019882
0.8704238820321134
0.8660486177088957
0.8788790401661207
0.875919163980539
0.8800155547181596
0.880412402019882
0.8800694028590277
0.8733056643209233
0.8831647294669978
0.8795372564255407
0.8765213754816862
0.8780181722940377
0.8678229772642311
0.

0.8789808636181284
0.8765262767297441
0.8653660564561766
0.8799199102304737
0.8756248679022375
0.8790196322179534
0.8789808636181284
0.8801758538571458
0.8768703162969266
0.879033540449364
0.8792852833452378
0.8801119361044365
0.8790441092898714
0.866800566989668
0.8452081803423047
0.8795461162685111
0.8759256679033594
0.8790196322179534
0.8789808636181284
0.8765262767297441
0.8653660564561766
0.8799199102304737
0.8756248679022375
0.8790196322179534
0.8789808636181284
0.8801758538571458
0.8768703162969266
0.879033540449364
0.8792852833452378
0.8801119361044365
0.8790441092898714
0.866800566989668
0.8452081803423047
0.8795461162685111
0.8759256679033594
0.8790196322179534
0.8789808636181284
0.8765262767297441
0.8653660564561766
0.8799199102304737
0.8756248679022375
0.8790196322179534
0.8789808636181284
0.8801758538571458
0.8768703162969266
0.879033540449364
0.8792852833452378
0.8801119361044365
0.8790441092898714
0.866800566989668
0.8452081803423047
0.8795461162685111
0.8759256679033594

0.8658528119996753
0.8800932872181253
0.8760968822589802
0.8801825080684846
0.8791919931087611
0.880718230162895
0.876948262948123
0.8802047535738398
0.8804178214438819
0.8806232447141867
0.8790242406846671
0.867866603940593
0.8449792941896686
0.8811910657085454
0.8761555549992893
0.8801825080684846
0.8791919931087611
0.8769664607751203
0.8658528119996753
0.8800932872181253
0.8760968822589802
0.8801825080684846
0.8791919931087611
0.880718230162895
0.876948262948123
0.8802047535738398
0.8804178214438819
0.8806232447141867
0.8790242406846671
0.867866603940593
0.8449792941896686
0.8811910657085454
0.8761555549992893
0.8801825080684846
0.8791919931087611
0.8769664607751203
0.8658528119996753
0.8800932872181253
0.8760968822589802
0.8801825080684846
0.8791919931087611
0.880718230162895
0.876948262948123
0.8802047535738398
0.8804178214438819
0.8806232447141867
0.8790242406846671
0.867866603940593
0.8449792941896686
0.8811910657085454
0.8761555549992893
0.8801825080684846
0.8791919931087611
0.

0.8809623532269896
0.8801598709746772
0.8799493469861076
0.8771239659731854
0.8811567155493943
0.8796863990343374
0.8812023596600973
0.8791582862488098
0.8718969761269676
0.8479537680198604
0.8780558191049437
0.8737285697193572
0.8809623532269896
0.8801598709746772
0.8757822451458739
0.8693022544401164
0.8789255101207968
0.875854730843722
0.8809623532269896
0.8801598709746772
0.8799493469861076
0.8771239659731854
0.8811567155493943
0.8796863990343374
0.8812023596600973
0.8791582862488098
0.8718969761269676
0.8479537680198604
0.8780558191049437
0.8737285697193572
0.8809623532269896
0.8801598709746772
0.8757822451458739
0.8693022544401164
0.8789255101207968
0.875854730843722
0.8809623532269896
0.8801598709746772
0.8799493469861076
0.8771239659731854
0.8811567155493943
0.8796863990343374
0.8812023596600973
0.8791582862488098
0.8734958851514998
0.8694122503457332
0.8786079901822553
0.875373982909562
0.8784134991360747
0.8761096502717663
0.877237198306731
0.8738151759201305
0.88000506292598

...

0.8811841501484793
0.8810835612438476
0.8806552639330083
0.8698240693564897
0.8465413297277453
0.8791529152898866
0.8760828404683911
0.8802610843044069
0.8805300595054453
0.8766412265426372
0.8663079663458148
0.8797785236459226
0.8763314846026494
0.8802610843044069
0.8805300595054453
0.8806009491612093
0.8769870404960495
0.8801984604562918
0.8811841501484793
0.8810835612438476
0.8806552639330083
0.8698240693564897
0.8465413297277453
0.8791529152898866
0.8760828404683911
0.8802610843044069
0.8805300595054453
0.8766412265426372
0.8663079663458148
0.8797785236459226
0.8763314846026494
0.8802610843044069
0.8805300595054453
0.8806009491612093
0.8769870404960495
0.8801984604562918
0.8811841501484793
0.8810835612438476
0.8806552639330083
0.8698240693564897
0.8465413297277453
0.8791529152898866
0.8760828404683911
0.8802610843044069
0.8805300595054453
0.8766412265426372
0.8663079663458148
0.8797785236459226
0.8763314846026494
0.8802610843044069
0.8805300595054453
0.8806009491612093
0.8769870404

0.8809990510903699
0.8808281339122991
0.8696388894012973
0.8476244534165592
0.8795281498083393
0.8759348029661261
0.8802148976136399
0.8808586540234873
0.8770414508020781
0.8666398074223484
0.8801362828449362
0.8764749309309764
0.8802148976136399
0.8808586540234873
0.8809533956210811
0.876787006104152
0.8808309906242858
0.8812414792260244
0.8809990510903699
0.8808281339122991
0.8696388894012973
0.8476244534165592
0.8795281498083393
0.8759348029661261
0.8802148976136399
0.8808586540234873
0.8770414508020781
0.8666398074223484
0.8801362828449362
0.8764749309309764
0.8802148976136399
0.8808586540234873
0.8809533956210811
0.876787006104152
0.8808309906242858
0.8812414792260244
0.8809990510903699
0.8808281339122991
0.8696388894012973
0.8476244534165592
0.8795281498083393
0.8759348029661261
0.8802148976136399
0.8808586540234873
0.8770414508020781
0.8666398074223484
0.8801362828449362
0.8764749309309764
0.8802148976136399
0.8808586540234873
0.8809533956210811
0.876787006104152
0.8808309906242

0.8725933741413959
0.8475967749071498
0.8789961678414947
0.8747456105568348
0.8816258173362245
0.8792536150476856
0.8771174708245351
0.8677657822117487
0.878824818772904
0.8747353473213336
0.8816258173362245
0.8792536150476856
0.881084800658002
0.8774825888328818
0.8798336317194909
0.8794109207346993
0.8805170769036171
0.8795324285843904
0.8761521831798383
0.8696811168507669
0.877822123957425
0.8772370760586088
0.8788948418419992
0.8777694310427301
0.8772487491813789
0.8747430682058971
0.8787453293249466
0.8779979970841604
0.8788948418419992
0.8777694310427301
0.8789783896737103
0.8773390274240356
0.8796370926377391
0.8796238416909393
0.8795825931193932
0.8776499766037263
0.869925214111467
0.851631581303486
0.8796911745743999
0.8756075008638039
0.8807622017531428
0.8810683595289222
0.8771279256025807
0.8664984760164076
0.8797545010155163
0.8765575351903835
0.8807622017531428
0.8810683595289222
0.8810276044849846
0.8769419436905315
0.8812624163018258
0.8812299984690961
0.880982400796120

0.8787633100591847
0.8747881402101125
0.8813820922940817
0.87925257961083
0.8773399235349792
0.8682603976267305
0.8789052592690613
0.8751518852931901
0.8813820922940817
0.87925257961083
0.8813965375747502
0.8772044769467305
0.8796509611834225
0.8793451024741105
0.8803353499276102
0.8792680527268842
0.8726792425277156
0.847153773091172
0.8788315310635118
0.8748707424783918
0.8813514550391797
0.879324746047416
0.8772876099882495
0.8678406187519107
0.8788504561819322
0.8752340733030272
0.8813514550391797
0.879324746047416
0.8812982065223288
0.8775404451172344
0.8796183772798708
0.8791465553439562
0.8803353499276102
0.879299146076875
0.8726792425277156
0.847153773091172
0.8788315310635118
0.8748707424783918
0.8813514550391797
0.879324746047416
0.8772876099882495
0.8678406187519107
0.8788504561819322
0.8752340733030272
0.8813514550391797
0.879324746047416
0.8812982065223288
0.8775404451172344
0.8796183772798708
0.8791465553439562
0.8803353499276102
0.879299146076875
0.8726792425277156
0.847

0.8807840156133588
0.8770427185508488
0.8664562166846707
0.8797861025955555
0.876418585189688
0.8807351354138926
0.8807840156133588
0.8813101014036088
0.8766825070339435
0.8812114079322678
0.8814555550269677
0.8812762883831956
0.8808236114329623
0.8690943640681104
0.8477504340058601
0.8798157508427558
0.8756436651713871
0.8807351354138926
0.8807840156133588
0.8770427185508488
0.8664562166846707
0.8797861025955555
0.876418585189688
0.8807351354138926
0.8807840156133588
0.8813101014036088
0.8766825070339435
0.8812114079322678
0.8814555550269677
0.8812762883831956
0.8808236114329623
0.8690943640681104
0.8477504340058601
0.8798157508427558
0.8756436651713871
0.8807351354138926
0.8807840156133588
0.8770427185508488
0.8664562166846707
0.8797861025955555
0.876418585189688
0.8807351354138926
0.8807840156133588
0.8813101014036088
0.8766825070339435
0.8812114079322678
0.8814555550269677
0.8812762883831956
0.8808236114329623
0.8690943640681104
0.8477504340058601
0.8798157508427558
0.8756436651713

In [31]:
combination_grid.to_csv("parameter_tuning_combination_grid.csv")
combination_grid

,n_estimators,max_features,max_depth,min_samples_split,min_samples_leaf,bootstrap,avg_auc
0,40,auto,10,2,1,True,0.878399
1,40,auto,10,2,1,False,0.869770
2,40,auto,10,2,2,True,0.877154
3,40,auto,10,2,2,False,0.873489
4,40,auto,10,2,4,True,0.876220
...,...,...,...,...,...,...,...
7717,1000,log2,110,10,1,False,0.877308
7718,1000,log2,110,10,2,True,0.879893
7719,1000,log2,110,10,2,False,0.879646
7720,1000,log2,110,10,4,True,0.880483


In [36]:
# best parameters
combination_grid.sort_values(by='avg_auc', ascending=False).head(20)

,n_estimators,max_features,max_depth,min_samples_split,min_samples_leaf,bootstrap,avg_auc
482,40,log2,50,10,2,True,0.883165
590,40,log2,110,10,2,True,0.883165
446,40,log2,30,10,2,True,0.883165
500,40,log2,60,10,2,True,0.883165
518,40,log2,70,10,2,True,0.883165
572,40,log2,100,10,2,True,0.883165
536,40,log2,80,10,2,True,0.883165
428,40,log2,20,10,2,True,0.883165
554,40,log2,90,10,2,True,0.883165
464,40,log2,40,10,2,True,0.883165


In [42]:
### prediction 1
# use the best_random parameters to train the whole dataset
from sklearn.ensemble import RandomForestClassifier

rf1 = RandomForestClassifier( n_estimators=40, max_features='log2', max_depth=50, min_samples_split=10, min_samples_leaf=2, bootstrap=True, random_state = 2345)
rf1.fit(df_im_input, df_im_output)

RandomForestClassifier(max_depth=50, max_features='log2', min_samples_leaf=2,
                       min_samples_split=10, n_estimators=40,
                       random_state=2345)

In [43]:
# predict on the test set
test_input=combined.iloc[891:]

test_preds=rf1.predict(test_input)
test_df['Survived'] = test_preds

In [44]:
test_submission = test_df.loc[:,["PassengerId", "Survived"]].to_csv("submission_rfcv1.csv", index=False)
ts = pd.read_csv("submission_rfcv1.csv")
ts

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [48]:
### prediction 2

# use the best_random parameters to train the whole dataset
from sklearn.ensemble import RandomForestClassifier

rf2 = RandomForestClassifier( n_estimators=40, max_features='log2', max_depth=110, min_samples_split=10, min_samples_leaf=2, bootstrap=True, random_state = 2345)
rf2.fit(df_im_input, df_im_output)

# predict on the test set
test_preds=rf2.predict(test_input)
test_df['Survived'] = test_preds

# test submission
test_submission = test_df.loc[:,["PassengerId", "Survived"]].to_csv("submission_rfcv2.csv", index=False)
ts2 = pd.read_csv("submission_rfcv2.csv")
# ts2

In [47]:
### prediction 3

# use the best_random parameters to train the whole dataset
from sklearn.ensemble import RandomForestClassifier

rf3 = RandomForestClassifier( n_estimators=40, max_features='auto', max_depth=10, min_samples_split=10, min_samples_leaf=2, bootstrap=False, random_state = 2345)
rf3.fit(df_im_input, df_im_output)

# predict on the test set
test_preds=rf3.predict(test_input)
test_df['Survived'] = test_preds

# test submission
test_submission = test_df.loc[:,["PassengerId", "Survived"]].to_csv("submission_rfcv3.csv", index=False)
ts3 = pd.read_csv("submission_rfcv2.csv")
# ts2

#### Conclusion
Predcition 1 and 2 are scoring 0.78708 in Kaggle, which is an improvement compared to RandomizedSearchCV package.  
```RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=2, min_samples_split=10, n_estimators=1680)```

In [52]:
print(rf1)
print(rf2)

RandomForestClassifier(max_depth=50, max_features='log2', min_samples_leaf=2,
                       min_samples_split=10, n_estimators=40,
                       random_state=2345)
RandomForestClassifier(max_depth=110, max_features='log2', min_samples_leaf=2,
                       min_samples_split=10, n_estimators=40,
                       random_state=2345)
